In [411]:
import pandas as pd
import numpy as np
import seaborn as sn
import tensorflow as tf
import matplotlib.pyplot as plt
from math import log
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn import neighbors, metrics

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from time import sleep


## Betting odds Dataframe

Source: <a href="http://tennis-data.co.uk/" target="_blank">http://tennis-data.co.uk/</a>

In [468]:
df_betting=pd.DataFrame()
for i in range(2002,2021):
    
    url="http://tennis-data.co.uk/%d/%d.xlsx" %(i,i)
    df=pd.read_excel(url)
    df_betting= pd.concat([df_betting,df],ignore_index=True)

Create a datframe with Atp matches since 2002

In [413]:
def checkName(Name):
    if '..' in Name:
        Name=Name.replace('..','.')
    else:
        Name=Name
        
    if '. ' in Name:
        Name=Name.replace('. ','.')
    else:
        Name=Name    
        
    length=len(Name)
    if Name[length-1]!='.':
        Name=Name+'.'
    else:
        Name=Name
    return Name

In [414]:
df_betting_players=pd.DataFrame()
#Keep same syntax for player Name in order to have an unique ID for each player
df_betting_players["Winner"]=df_betting["Winner"].apply(lambda x: checkName(x))
df_betting_players["Loser"]=df_betting["Loser"].apply(lambda x: checkName(x))

## Players Dataframe

Source: <a href="https://www.kaggle.com/romanzdk/atp-players-overviews" target="_blank">https://www.kaggle.com/romanzdk/atp-players-overviews/</a>

In [415]:
df_players=pd.read_csv('player_overviews.csv')
df_players.head()

,a002,ricardo-acuna,Ricardo,Acuna,http://www.atpworldtour.com/en/players/ricardo-acuna/a002/overview,CHI,"Jupiter, FL, USA","Santiago, Chile",1958.01.13,1958,01,13,0,150,68,"5'9""",69,175,Unnamed: 18,Unnamed: 19
0,a001,sadiq-abdullahi,Sadiq,Abdullahi,http://www.atpworldtour.com/en/players/sadiq-a...,NGR,NaN,NaN,1960.02.02,1960.0,2.0,2.0,0.0,0.0,0.0,"0'0""",0.0,0.0,NaN,NaN
1,a005,nelson-aerts,Nelson,Aerts,http://www.atpworldtour.com/en/players/nelson-...,BRA,NaN,"Cachoeira Do Sul, Brazil",1963.04.25,1963.0,4.0,25.0,0.0,165.0,75.0,"6'2""",74.0,188.0,NaN,NaN
2,a004,egan-adams,Egan,Adams,http://www.atpworldtour.com/en/players/egan-ad...,USA,"Palmetto, FL, USA","Miami Beach, FL, USA",1959.06.15,1959.0,6.0,15.0,0.0,160.0,73.0,"5'10""",70.0,178.0,NaN,NaN
3,a006,ronald-agenor,Ronald,Agenor,http://www.atpworldtour.com/en/players/ronald-...,USA,"Beverly Hills, California, USA","Rabat, Morocco",1964.11.13,1964.0,11.0,13.0,1983.0,180.0,82.0,"5'11""",71.0,180.0,NaN,NaN
4,a007,juan-aguilera,Juan,Aguilera,http://www.atpworldtour.com/en/players/juan-ag...,ESP,NaN,"Barcelona, Spain",1962.03.22,1962.0,3.0,22.0,0.0,150.0,68.0,"6'0""",72.0,183.0,NaN,NaN


In [416]:
df_players=pd.read_csv('player_overviews.csv')
df_players=df_players.drop(df_players.columns[4:17],axis=1)
df_players=df_players.drop(df_players.columns[-1],axis=1)
df_players.columns=['AtpId','AtpName','FirstName','Name','Height','Handedness']
df_players.head()

,AtpId,AtpName,FirstName,Name,Height,Handedness
0,a001,sadiq-abdullahi,Sadiq,Abdullahi,0.0,NaN
1,a005,nelson-aerts,Nelson,Aerts,188.0,NaN
2,a004,egan-adams,Egan,Adams,178.0,NaN
3,a006,ronald-agenor,Ronald,Agenor,180.0,NaN
4,a007,juan-aguilera,Juan,Aguilera,183.0,NaN


### Change Name syntax to be similar to betting dataframe

For a future merge with the betting dataframe, we need to apply the same syntax to the Name column

In [417]:
def PlayerName(Name,FirstName):
    length= len(Name)
    if "-" in FirstName:
        FirstName=FirstName.split("-")
        player=Name+' '+FirstName[0][0]+'.'+FirstName[1][0]+'.'
    elif " " in FirstName:
        FirstName=FirstName.split()
        if len(FirstName)>1:
            player=Name+' '+FirstName[0][0]+'.'+FirstName[1][0]+'.'
        else:
            player=Name+' '+FirstName[0]+'.'
    elif Name[length-1]=='.':
        player=Name+FirstName[0]+'.'
        
    else:
        player=Name+' '+FirstName[0]+'.'
    return player.title()

In [418]:
df_players['NewName']=df_players.apply(lambda x: PlayerName(x["Name"],x["FirstName"]), axis=1)

Some players have their name and first letter of their firstname similar. We have to distinguished them 

In [419]:
df_players=df_players.sort_values(["Name","FirstName"]).reset_index(drop=True)

In [420]:
def duplicatedNames(df):
    column=[df.loc[0,"NewName"]]
    for i in range(1,len(df)):
        j=0
        while df.loc[i,"NewName"]in column:
            df.loc[i,"NewName"]=df.loc[i,"Name"]+' '+df.loc[i,"FirstName"][0:j+1]+'.'
            j+=1
        
        column.append(df.loc[i,"NewName"])
    return column

In [421]:
df_players["NewName"]=duplicatedNames(df_players)

In [422]:
df_players=df_players.drop(["FirstName"],axis=1) #column not needed anymore
df_players=df_players.drop(["Name"],axis=1) #column not needed anymore
df_players.rename(columns={'NewName':'Name'}, inplace=True) #column renamed
df_players.head()

,AtpId,AtpName,Height,Handedness,Name
0,a451,bob-abbott,0.0,NaN,Abbott B.
1,a755,hashim-abdal,0.0,NaN,Abdal H.
2,ad60,robert-abdesselam,NaN,NaN,Abdesselam R.
3,a742,ahmed-abdrabuh,0.0,NaN,Abdrabuh A.
4,a706,syrym-abdukhalikov,0.0,NaN,Abdukhalikov S.


In [423]:
df_players[df_players["Name"].str.contains("Kuznetsov")]

,AtpId,AtpName,Height,Handedness,Name
5378,k737,alex-kuznetsov,183.0,Right-Handed,Kuznetsov A.
5379,kb54,andrey-kuznetsov,185.0,Right-Handed,Kuznetsov An.


In [424]:
print(df_players.shape)

(10911, 5)


### Select only commun players

In [425]:
df_players=df_players[df_players["Name"].isin(df_betting_players['Winner'])| df_players["Name"].isin(df_betting_players['Loser'])] 

In [426]:
print(df_players.shape)

(1138, 5)


### Update players dataframe

The dataset is not udpated and some current players are missing. The idea is to scrap datas from the https://www.atptour.com/ website.

In [427]:
#Player informations' are accessible through their atp overview webpage. 
#To get this page we need the name and the id given by the website.
#A first scrap on 'https://www.atptour.com/en/rankings/singles/ gives the url of the overview page
#Then for each player, we scrap First name and Last name in their overview webpage and create a dictionnary

def AtpPlayers():
    AtpIdList=[]
    AtpNameList=[]
    FirstNameList=[]
    LastNameList=[]
    ActualRankingList=[]
    atpRank='https://www.atptour.com/en/rankings/singles/?rankDate=2020-9-14&countryCode=all&rankRange=0-700'
    req = Request(atpRank, headers={'User-Agent': ''})
    webpageAtp = urlopen(req).read()
    soup = BeautifulSoup(webpageAtp, 'html.parser')
    for i in range(0,700):
        url_overview='https://www.atptour.com' + soup.select('.player-cell a')[i]['href']
        AtpName=url_overview.split("/")[5]
        AtpId=url_overview.split("/")[6]
        
        req2 = Request(url_overview, headers={'User-Agent': 'Mozilla/5.0'})
        webpage_overview = urlopen(req2).read()
        soup2 = BeautifulSoup(webpage_overview, 'html.parser')
        try:
            FirstName = soup2.find( attrs={'class': 'first-name'}).get_text()
        except AttributeError:
            FirstName=np.nan
        
        try:
            Name = soup2.find( attrs={'class': 'last-name'}).get_text()
        except AttributeError:
            Name=np.nan
            
        
            
        AtpIdList.append(AtpId)
        AtpNameList.append(AtpName)
        FirstNameList.append(FirstName)
        LastNameList.append(Name)
        ActualRankingList.append(i+1)
        
        Dictionnary={'AtpId':AtpIdList,'AtpName':AtpNameList,'FirstName':FirstNameList,'Name':LastNameList,'ActualRanking':ActualRankingList}

    return Dictionnary

In [428]:
df_AtpPlayers=AtpPlayers() #get a dictionnary with scraped datas
df_AtpPlayers=pd.DataFrame(df_AtpPlayers) #convert dictionnary into dataframe
df_AtpPlayers['Name']=df_AtpPlayers.apply(lambda x: PlayerName(x["Name"],x["FirstName"]), axis=1)
df_AtpPlayers=df_AtpPlayers.drop(["FirstName"],axis=1) #column not needed anymore

In [429]:
df_total_players=pd.concat([df_players, df_AtpPlayers], ignore_index=True) # concat the scraping dataframe with the original one
df_total_players.drop_duplicates(subset ="AtpId",keep = "last", inplace = True) # KEEP UNIQUE VALUES
print(df_total_players.shape)
df_total_players.head()

(1520, 6)


,AtpId,AtpName,Height,Handedness,Name,ActualRanking
0,a618,mohammed-abdulla,0.0,NaN,Abdulla M.,NaN
1,a391,maximilian-abel,0.0,NaN,Abel M.,NaN
2,a389,jose-acasuso,191.0,Right-Handed,Acasuso J.,NaN
3,a305,jacob-adaktusson,193.0,NaN,Adaktusson J.,NaN
4,a310,emin-agaev,0.0,NaN,Agaev E.,NaN


In [430]:
df_total_players[df_total_players["Name"]=="Nadal R."]

,AtpId,AtpName,Height,Handedness,Name,ActualRanking
1139,n409,rafael-nadal,NaN,NaN,Nadal R.,2.0


Height, Handedness and Backhand are interesting criterias to analyse but we saw in the model prediction notebook that backhand had too many 'Nan' values even after scraping. Thanks to the AtId and AtpNAme we scraped on the website https://www.atptour.com/ 

In [431]:
def scraping(atpId,atpName,height,handedness):
    
    try:
        url_atp = 'https://www.atptour.com/en/players/'+atpName+'/'+atpId+'/overview'
        req = Request(url_atp, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'html.parser')

        if ((height!=height) or (height==0.0)): #check NaN value or height equals to 0
            try:
                height = soup.find( attrs={'class': 'table-height-cm-wrapper'}).get_text()
                height= height.replace('(','').replace(')','').replace('cm','')
                if str(height)=='0':
                    height=np.nan
                else:
                    height=height
            except AttributeError:
                height=np.nan
        else:
            height=height

        if handedness!=handedness: #check NaN value
            try:
                plays= soup.find_all( attrs={'class':"table-value"})
                if len(plays)>=2:
                    plays=plays[2].get_text()
                    if len(plays)>1:
                        plays=plays.split(", ")

                        handedness=plays[0].replace("\r","")
                        handedness=handedness.replace("\n","")  
                    else:
                        handedness=plays.replace("\r","")
                        handedness=handedness.replace("\n","")
                    if handedness=='':
                        handedness=np.nan
                    else:
                        handedness=handedness
                else:
                    handedness=np.nan
            except AttributeError:
                handedness=np.nan
        else:
            handedness=handedness

        
    except:
        height=handedness=np.nan
    
    
    
    return height,handedness

In [432]:
def UpdateRanking(atpId,atpName):
    url_atp = 'https://www.atptour.com/en/players/'+atpName+'/'+atpId+'/overview'
    req = Request(url_atp, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    try:
        Ranking = soup.find( attrs={'class': 'data-number'}).get_text()
    except:
        Ranking=np.nan
    try:
        int(Ranking)
        return Ranking
    except ValueError: #For inactive players 
        return np.nan

In [433]:
df_total_players.dtypes

AtpId             object
AtpName           object
Height           float64
Handedness        object
Name              object
ActualRanking    float64
dtype: object

In [434]:
df_total_players[['Height','Handedness']]=df_total_players.apply(lambda x : scraping(x['AtpId'],x['AtpName'],x['Height'],x['Handedness']),axis=1, result_type="expand")
df_total_players.head()

,AtpId,AtpName,Height,Handedness,Name,ActualRanking
0,a618,mohammed-abdulla,NaN,NaN,Abdulla M.,NaN
1,a391,maximilian-abel,NaN,Right-Handed,Abel M.,NaN
2,a389,jose-acasuso,191,Right-Handed,Acasuso J.,NaN
3,a305,jacob-adaktusson,193,Right-Handed,Adaktusson J.,NaN
4,a310,emin-agaev,NaN,Right-Handed,Agaev E.,NaN


In [467]:
df_total_players["Height"] = df_total_players.Height.astype(float) # convert to float

In [435]:
df_total_players['Id']=df_total_players.index

In [436]:
df_copy=df_total_players.copy()

In [437]:
df_total_players.head()

,AtpId,AtpName,Height,Handedness,Name,ActualRanking,Id
0,a618,mohammed-abdulla,NaN,NaN,Abdulla M.,NaN,0
1,a391,maximilian-abel,NaN,Right-Handed,Abel M.,NaN,1
2,a389,jose-acasuso,191,Right-Handed,Acasuso J.,NaN,2
3,a305,jacob-adaktusson,193,Right-Handed,Adaktusson J.,NaN,3
4,a310,emin-agaev,NaN,Right-Handed,Agaev E.,NaN,4


In [438]:
df_total_players[df_total_players["Name"]=="Nadal R."]

,AtpId,AtpName,Height,Handedness,Name,ActualRanking,Id
1139,n409,rafael-nadal,185,Left-Handed,Nadal R.,2.0,1139


## Model Dataframe

### Clean Dataframe

We can notice that depending the year, Betting Broker are not the same. So it's better to use the Min/Max and Average of betting odds. 

In [469]:
#Fill NaN values
df_betting['MaxW']=df_betting['MaxW'].fillna(round(df_betting[['B365W', 'CBW', 'GBW', 'IWW','SBW', 'B&WW', 'EXW', 'PSW', 'UBW', 'LBW', 'SJW']].max(axis=1),2))
df_betting['MaxL']=df_betting['MaxL'].fillna(round(df_betting[['B365L', 'CBL', 'GBL', 'IWL','SBL', 'B&WL', 'EXL', 'PSL', 'UBL', 'LBL', 'SJL']].max(axis=1),2))
df_betting['AvgW']=df_betting['AvgW'].fillna(round(df_betting[['B365W', 'CBW', 'GBW', 'IWW','SBW', 'B&WW', 'EXW', 'PSW', 'UBW', 'LBW', 'SJW']].mean(axis=1),2))
df_betting['AvgL']=df_betting['AvgL'].fillna(round(df_betting[['B365L', 'CBL', 'GBL', 'IWL','SBL', 'B&WL', 'EXL', 'PSL', 'UBL', 'LBL', 'SJL']].mean(axis=1),2))

In [470]:
# drop useless datas
df_betting=df_betting[df_betting['MaxW'].notna() & df_betting['MaxL'].notna()] # select rows where betting odds are present
df_betting=df_betting.drop(df_betting[df_betting["Comment"]!="Completed"].index) #keep only completed matches
df_betting=df_betting.drop(['Comment','Date','ATP','Best of'],axis=1) # drop useless columns
df_betting=df_betting.drop(['B365W', 'CBW', 'GBW', 'IWW','SBW', 'B&WW', 'EXW', 'PSW', 'UBW', 'LBW', 'SJW','B365L', 'CBL', 'GBL', 'IWL','SBL', 'B&WL', 'EXL', 'PSL', 'UBL', 'LBL', 'SJL'],axis=1)
df_betting=df_betting.drop(['W1','L1','W2','L2','W3','L3','W4','L4','W5','L5','Wsets','Lsets','WPts','LPts'],axis=1)
df_betting=df_betting.drop(df_betting[(df_betting["WRank"]=='NR') |  (df_betting["LRank"]=='NR')].index) # drop matches with No ranked players
df_betting=df_betting.dropna()
df_betting["LRank"] = df_betting.LRank.astype(float)# no more 'NR' players so we can convert into float

### Preprocessing data

In [471]:
#Keep same syntax for player Name in order to have an unique ID for each player
df_betting["Winner"]=df_betting["Winner"].apply(lambda x: checkName(x))
df_betting["Loser"]=df_betting["Loser"].apply(lambda x: checkName(x))

In [472]:
#Some Series have been renamed over years
df_betting['Series'].replace({"International Gold":"ATP500","Masters Cup":"ATP Finals","Masters":"ATP Finals"},inplace =True)

In [473]:
#Rename columns: Winner -> Player1 & Loser -> Player2
df_betting.rename(columns={'Winner':'Player1','Loser':'Player2','WRank':'P1Rank','LRank':'P2Rank','MaxW':'MaxP1','MaxL':'MaxP2','AvgW':'AvgP1','AvgL':'AvgP2'}, inplace=True)
#Create the column for the label prediction: Player1 Win
df_betting['P1Winner']=1 

In the actual format all winners are Player 1, it will be a problem for the model.
So for random rows, Players 1 and Player 2 datas are switched

In [474]:
mask=df_betting.sample(frac=.5).index
df_betting.loc[mask,['Player1','Player2','P1Rank','P2Rank','MaxP1','MaxP2','AvgP1','AvgP2']] = df_betting.loc[mask,['Player2','Player1','P2Rank','P1Rank','MaxP2','MaxP1','AvgP2','AvgP1']].values
df_betting.loc[mask,['P1Winner']]=0

In [475]:
df_betting=df_betting.merge(df_total_players,left_on='Player1',right_on="Name", how='left',suffixes=['P2','P1']) #first merge for winner player
df_betting=df_betting.merge(df_total_players,left_on='Player2',right_on="Name", how='left',suffixes=['P1','P2'])#second merge for loser player

In [476]:
df_betting=df_betting.drop(['NameP1','NameP2',"AtpIdP1","AtpIdP2","ActualRankingP1","ActualRankingP2","AtpNameP1","AtpNameP2","Player1","Player2"],axis=1) # keep only the player ID
df_betting=df_betting.dropna() # delete rows with NaN values
df_betting=df_betting.drop_duplicates() #delete duplicated rows

In [477]:
df_betting.head(10)

,Location,Tournament,Series,Court,Surface,Round,P1Rank,P2Rank,MaxP1,MaxP2,AvgP1,AvgP2,P1Winner,HeightP1,HandednessP1,IdP1,HeightP2,HandednessP2,IdP2
0,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,46.0,25.0,2.20,1.60,2.17,1.55,0,185.0,Right-Handed,539.0,175.0,Left-Handed,34.0
1,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,9.0,309.0,1.15,4.25,1.15,4.25,1,185.0,Right-Handed,423.0,196.0,Right-Handed,321.0
2,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,57.0,10.0,3.80,1.36,3.09,1.30,1,188.0,Right-Handed,642.0,185.0,Right-Handed,886.0
3,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,80.0,93.0,1.50,2.55,1.47,2.35,1,183.0,Right-Handed,645.0,183.0,Right-Handed,904.0
4,Adelaide,AAPT Championships,International,Outdoor,Hard,1st Round,104.0,53.0,1.50,2.40,1.50,2.27,1,196.0,Right-Handed,792.0,191.0,Right-Handed,118.0
5,Adelaide,AAPT Championships,International,Outdoor,Hard,2nd Round,150.0,25.0,2.75,1.45,2.53,1.43,0,185.0,Right-Handed,647.0,175.0,Left-Handed,34.0
6,Adelaide,AAPT Championships,International,Outdoor,Hard,2nd Round,50.0,24.0,3.20,1.33,2.92,1.31,0,188.0,Left-Handed,368.0,191.0,Right-Handed,286.0
7,Adelaide,AAPT Championships,International,Outdoor,Hard,2nd Round,37.0,80.0,1.60,2.38,1.54,2.22,1,193.0,Right-Handed,596.0,183.0,Right-Handed,645.0
9,Adelaide,AAPT Championships,International,Outdoor,Hard,2nd Round,41.0,70.0,1.80,1.90,1.74,1.86,1,183.0,Right-Handed,1239.0,191.0,Right-Handed,171.0
10,Adelaide,AAPT Championships,International,Outdoor,Hard,2nd Round,89.0,57.0,2.75,1.40,2.62,1.38,0,191.0,Left-Handed,597.0,188.0,Right-Handed,642.0


Columns label encoding

In [478]:
def Labelizer(column):
    label=sorted(column.unique())
    dictionnary = { val : idx for idx,val in enumerate(label) }
    colLab=column.map(dictionnary)
    return colLab

In [481]:
df_betting=df_betting.apply(lambda x: Labelizer(x) if x.dtype==object else x,axis=0)